# Chapter 10 시각화

## 10.1 설명 가능한 인공지능 Explainable AI, XAI

### 10.1.1 Class Activation Map, CAM
- 이미지 분류 문제에서 이미지 내에서 어느 영역을 보고 클래스 분류를 결정했는지를 설명하는 시각화 방법
- 합성곱 신경망의 마지막 층에서 나온 피쳐맵과 분류기의 가중치를 이용해 영역을 찾아 내는 방식
- 합성곱 신경망의 마지막 층에서 나온 피쳐맵을 분류기에 넣기 위해 일렬로 펴는 Flatten을 해주는데 이때 객체 위치에 대한 정보가 소실되므로 CAM을 사용하기 위해선 합성곱 신경망의 마지막 층에서의 각각의 피쳐맵의 평균값을 사용하는 Gloabl Average Pooling, GAP을 사용

In [1]:
# 라이브러리 불러오기
# CIFAR10 이미지보다 사이즈가 큰 STL10 데이터를 사용함. CAM 가독성을 확보하기 위해 128x128로 늘림
# 비행기, 새, 자동차, 고양이, 사슴, 개, 말, 원숭이, 배, 트럭 클래스

import numpy as np
from matplotlib import pyplot as plt
import cv2
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

In [2]:
# 데이터 불러오기

transform = transforms.Compose([transforms.Resize(128), transforms.ToTensor()])
trainset = torchvision.datasets.STL10(root='./data',split='train',download=True,transform=transform) # 96x96
trainloader = torch.utils.data.DataLoader(trainset, batch_size=40, shuffle=True)
testset = torchvision.datasets.STL10(root='./data',split='test',download=True,transform=transform) # 96x96
testloader = torch.utils.data.DataLoader(testset, batch_size=40, shuffle=True)

100%|██████████| 2640397119/2640397119 [03:50<00:00, 11460024.73it/s]


Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# 모델 불러오기

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torchvision.models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs,10)
model = model.to(device)
model.load_state_dict(torch.load('./models/stl10_resnet18.pth'))

In [ ]:
# 정확도 확인하기
# 학습 정확도가 99%, 평가 정확도가 86%인 모델을 이용하여 CAM 결과 산출

def acc(dataloader):
  correct = 0
  total = 0
  model.eval()
  with torch.no_grad():
    for data in dataloader:
      images, labels = data[0].to(device), data[1].to(device)
      outputs = model(images)
      _,predicted = torch.max(outputs.detach(),1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  print('Accuracy: %d %%' % (100 * correct / total))
acc(trainloader)
acc(testloader)

In [ ]:
# CAM 구축하기

activation = {}
# 모델의 특정 레이어에서의 피쳐맵을 추출할 수 있도록 도와주는 역할
def get_activation(name):
  def hook(model,input,output):
    activation[name] = output.detach()
  return hook

In [ ]:
def cam(dataset,img_sample,img_size): # 데이터셋, 이미지 번호, 사이즈
  model.eval()
  with torch.no_grad():
    # ResNet18의 마지막 합성곱 층의 이름은 model.layer4[1].bn2이며, register_forward_hook을 이용하여 마지막 합성곱 층의 피쳐맵을 불러올 수 있도록 지정
    model.layer4[1].bn2.register_forward_hook(get_activation('final')) # 마지막 합성곱 층
    # CAM은 이미지를 하나씩 받아 클래스 별로 가중치와 피쳐맵을 곱하기에 이미지 하나를 불러옴
    data,label = dataset[img_sample]
    # 이미지 한 장은 3차원 이미지고, 모델의 입력 데이터는 배치사이즈를 포함하여 4차원 요구
    data.unsqueeze_(0) # 0번재 차원 하나 늘려줌(죽, [피쳐수,너비,높이] -> [1,피쳐수,너비,높이])
    # 모델의 예측값 구함
    output = model(data.to(device))
    _,prediction = torch.max(output,1)
    # 마지막 합성곱 층의 피쳐맵을 불러오고 분류기의 가중치 불러옴
    act = activation['final'].squeeze()
    w = model.fc.weight
    # 피쳐맵과 해당 예측 클래스와 관련된 가중치를 곱하여 누적
    for idx in range(act.size(0)):
      if idx == 0:
        tmp = act[idx] * w[prediction.item()][idx]
      else:
        tmp += act[idx] * w[prediction.item()][idx]
    # 계산된 CAM 이미지를 0~255 값으로 변환
    normalized_cam = tmp.cpu().numpy()
    normalized_cam = (normalized_cam - np.min(normalized_cam)) / (np.max(normalized_cam) - np.min(normalized_cam))
    # 원본 이미지 불러옴
    original_img = np.uint8(data[0][0] / 2 + 0.5) * 255
    # CAM 이미지를 원본 이미지와 동일한 크기로 리사이즈
    cam_img = cv2.resize(np.uint8(normalized_cam * 255), dsize=(img_size,img_size))
  return cam_img, original_img, prediction, label

In [5]:
# CAM 결과 산출 함수 정의하기

def plot_cam(dataset,img_size,start): # 데이터셋, 이미지 크기, 이미지 시작번호
  end = start + 20 # 시작 번호로부터 20장의 CAM 출력
  # 표 및 부분 그래프에 대한 크기와 설정 및 공백 조절
  fig, axs = plt.subplots(2, (end - start + 1) // 2, figsize=(20,4))
  fig.subplots_adjust(hspace=.01, wspace=.01)
  axs = axs.ravel()
  # 클래스 명 정의
  cls = ['airplane','bird','car','cat','deer','dog','horse','monkey','ship','truck']
  # 이미지를 하나씩 불러옴
  for i in range(start, end):
    cam_img, original_img, prediction, label = cam(dataset,i,img_size)
    # 정확한 예측은 라벨의 배경이 흰색, 잘못된 예측에 대해서는 빨간색으로 지정
    if prediction == label:
      color = 'white'
    else:
      color = 'red'
    # 원본 이미지는 흑백으로 바꾸고 CAM 결과는 jet을 이용해 히트맵으로 표현 (alpha: 히트맵 밝기값)
    axs[i - start].imshow(original_img, cmap='gray')
    axs[i - start].imshow(cam_img, cmap='jet', alpha=.4)
    # 라벨 주석 달아주고, 좌표 표시 모두 없앰
    axs[i - start].text(5, 5, cls[prediction], bbox={'facecolor':color,'pad':5})
    axs[i - start].axis('off')
  plt.show()

In [ ]:
# CAM 결과 산출
# 히트맵에서 빨간 부분은 결과에 크게 영향을 미쳤다는 의미, 파란색에 가까울수록 예측이 덜 영향 주는 영역

plot_cam(trainset, 128, 10)
plot_cam(testset, 128, 10)

## 10.2 차원 축소 기법
- 우리가 시각적으로 표현할 수 있는 차원은 3차원 이하이므로 고차원의 벡터들을 3차원 이하의 저차원으로 바꿔야 함ㄴ

### 10.2.1 t-distributed Stochastic Neighbor Embedding
- 합성곱 신경망을 거쳐 나온 고차원의 피쳐맵을 분석하기 위해 사용되는 차원 축소 기법 중 하나

In [ ]:
# 라이브러리 불러오기

from sklearn.manifold import TSNE
import numpy as np
from matplotlib import pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn

In [ ]:
# 데이터 불러오기

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
testset = torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)
testloader = torch.utils.data.DataLoader(testset,batch_size=16)

In [ ]:
# 모델 불러오기
# 일반적으로 마지막 합성곱 층에서 추출된 피쳐맵을 가지고 분포를 그림
# hook을 사용하여 피쳐맵을 추출할 수도 있지만 코드 간소화를 위해 분류기 자체를 항등 함수 f(x)=x로 변경하여 모델에서의 출력값을 피쳐맵으로 뽑아낼 수 있음

# 항등 함수
class Identify(nn.Module):
  def __init__(self):
    super(Identify,self).__init__()
  def forward(self,x):
    return x

model = torchvision.models.resnet18(pretrained=False)
model.conv1 = nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs,10)
model = model.to(device)
model.load_state_dict(torch.load('./models/cifar10_resnet18.pth'))
model.fc = Identify()

In [ ]:
# 피쳐맵과 예측값 저장하기

actual = []
deep_features = []
model.eval()
with torch.no_grad():
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    # classifier를 제거했기 때문에 이전 값인 Global Average Pooling값이 features가 됨 (즉, 이미지의 피쳐 크기는 512이므로 512차원 벡터가 됨)
    features = model(images)
    deep_features += features.cpu().tolist()
    actual += labels.cpu().tolist()

In [ ]:
# t-SNE 정의하기

# n_components: 차원 축소의 차원 수. 512차원의 모든 deep_features 값들을 2차원 좌표로 축소한다는 의미
# 또한 t-SNE는 차원 축소 시 임의의 점을 기준으로 잡고 저차원 임베딩을 함
tsne = TSNE(n_components=2, random_state=0)
# 차원 축소 데이터 만듦. cluster는 각 이미지에 대응하는 2차원 벡터들의 모임
# cluster의 0열은 x좌표, 1열은 y좌표가 됨
cluster = np.array(tsne.fit_transform(np.array(deep_features)))
actual = np.array(actual)

In [ ]:
# t-SNE 그래프 그리기

plt.figure(figsize=(10,10))
cifar = ['plane','car','bird','cat','deer','dog','horse','monkey','ship','truck']
# 각 클래스를 하나씩 불러와서 scaatter 함수로 좌표를 찍어줌
# 실제값 actual을 0~9 차례대로 받아 for문이 한 번 돌 때마다 클래스 하나에 대한 데이터 그림
for i, label in zip(range(10), cifar):
  idx = np.where(actual == i)
  # cluster 좌표를 넣어주고 legend를 위해 label을 넣어줌
  plt.scatter(cluster[idx,0],cluster[idx,1],marker='.',label=label)
plt.legend()
plt.show()

10.2.2 주성분 분석 Principal Component Analysis, PCA
- 대표적 차원 축소기법 중 하나
- t-SNE의 차원 축소는 PCA보다 본래 특성을 덜 훼손하지만 차원 축소된 값을 활용하는데 한계가 있음. 하지만 PCA는 데이터의 고윳값을 이용해 분석하기에 클러스터링 및 데이터 분석에 활용도가 큼

In [ ]:
# PCA 정의하기

from sklearn.decomposition import PCA
pca = PCA(n_components=2)
cluster = np.array(pca.fit_transform(np.array(deep_features)))
print(pca.explained_variance_ratio_)

In [ ]:
# PCA 그래프 그리기

plt.figure(figsize=(10,10))

for i, label in zip(range(10), cifar):
  idx = np.where(actual == i)
  plt.scatter(cluster[idx,0],cluster[idx,1],marker='.',label=label)
plt.legend()
plt.show()